#### Importación base de datos https://github.com/albertogcmr/pymongo-project
Este repositorio de Alberto explica cómo

## Proyecto MongoDB

* Importar base de datos a MongoDB
* Conexión a la base de datos con Pymongo
* Extraccion de la base de datos con Pymongo, filtrando según intereses
* Limpieza de datos con Pandas
* Columna formato geopoint 
* Guardar como Json
* Importar la base de datos a MongoDB
* Crear geoindex 
* Conexión a la nueva base de datos con Pymongo, extracción de todo.
* Consultas geoespaciales sobre tu dataframe usando $near

In [1]:
from pymongo import MongoClient
import pandas as pd

In [2]:
# conexión con la base de datos de mongo

client = MongoClient("mongodb://localhost:27017/")
db = client.companies 

In [3]:
# Query para pedir todo
companies = db.companies.find()

In [4]:
data_companies = pd.DataFrame(companies)

In [5]:
data_companies.head(3)

,_id,name,permalink,crunchbase_url,homepage_url,blog_url,blog_feed_url,twitter_username,category_code,number_of_employees,...,video_embeds,screenshots,external_links,partners,founded_month,founded_day,deadpooled_month,deadpooled_day,deadpooled_url,ipo
0,52cdef7c4bab8bd675297d8b,AdventNet,abc3,http://www.crunchbase.com/company/adventnet,http://adventnet.com,,,manageengine,enterprise,600.0,...,[],"[{'available_sizes': [[[150, 94], 'assets/imag...",[],[],NaN,NaN,NaN,NaN,NaN,NaN
1,52cdef7c4bab8bd675297d8c,Zoho,abc4,http://www.crunchbase.com/company/zoho,http://zoho.com,http://blogs.zoho.com/,http://blogs.zoho.com/feed,zoho,software,1600.0,...,"[{'embed_code': '<object width=""430"" height=""2...",[],[{'external_url': 'http://www.online-tech-tips...,[],9.0,15.0,NaN,NaN,NaN,NaN
2,52cdef7c4bab8bd675297d8a,Wetpaint,abc2,http://www.crunchbase.com/company/wetpaint,http://wetpaint-inc.com,http://digitalquarters.net/,http://digitalquarters.net/feed/,BachelrWetpaint,web,47.0,...,[],"[{'available_sizes': [[[150, 86], 'assets/imag...",[{'external_url': 'http://www.geekwire.com/201...,[],10.0,17.0,NaN,NaN,NaN,NaN


In [6]:
#Ejemplo filtro de datos con PyMongo

acquisition = db.companies.find({
    "acquisition.price_amount":{
        "$gt":10000000}}, {
    "name":1,
    "acquisition.price_amount":1, "offices":1})

In [15]:
acquisition = pd.DataFrame(acquisition)

In [8]:
#creamos columna con el formato que te pide Mongo para crear el goeindex

def geopoint(x): 
    return {"type": "Point", "coordinates": [20,20]}

In [9]:
acquisition['geo']=acquisition['offices'].apply(geopoint)

In [10]:
acquisition.head()

,_id,name,acquisition,offices,geo
0,52cdef7c4bab8bd675297d8a,Wetpaint,{'price_amount': 30000000},"[{'description': '', 'address1': '710 - 2nd Av...","{'type': 'Point', 'coordinates': [20, 20]}"
1,52cdef7c4bab8bd675297d92,Flektor,{'price_amount': 20000000},"[{'description': None, 'address1': '8536 Natio...","{'type': 'Point', 'coordinates': [20, 20]}"
2,52cdef7c4bab8bd675297d90,Postini,{'price_amount': 625000000},"[{'description': None, 'address1': '959 Skyway...","{'type': 'Point', 'coordinates': [20, 20]}"
3,52cdef7c4bab8bd675297d9a,Helio,{'price_amount': 39000000},"[{'description': None, 'address1': '10960 Wils...","{'type': 'Point', 'coordinates': [20, 20]}"
4,52cdef7c4bab8bd675297d99,Lala,{'price_amount': 80000000},"[{'description': 'Lala Headquarters', 'address...","{'type': 'Point', 'coordinates': [20, 20]}"


In [11]:
#Importar archivo json a Mongo

# mongoimport --db companies --collection prueba --jsonArray oficinas.json

In [12]:
#Volvemos a conectarnos a la base de datos

db = client.companies

In [13]:
# Cogemos todos los datos

near = db.prueba.find()
df = pd.DataFrame(near)

In [14]:
#Seleccionamos un geopoint 
test = df.iloc[0, ]
test.principal_office

IndexError: single positional indexer is out-of-bounds

### Operador $near

In [ ]:
#Función para encontrar puntos cercanos con $near

def geonear(geopoint, maxdistance=1000):
    return db.prueba.find({
        "principal_office":{
            "$near":{
                "$geometry":geopoint,
                "$maxDistance":maxdistance
            }}})

In [17]:
geonear(test.principal_office, 100000).count()

NameError: name 'geonear' is not defined